### Step1: LIbrary 불러오기

In [80]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

import torch.nn.init

### Step2: GPU 설정하기

In [76]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)

if device == 'cuda':
    torch.cuda.manual_seed_all(777)

### Step3: 학습 Parameter 작성하기 

In [64]:
learning_rate = 0.001
training_epochs = 15
batch_size =100

### Step4: Dataset 가져오기 & DataLoader 만들기

In [69]:
mnist_train = datasets.MNIST(root='\MNIST_data/',
                            train=True,
                            transform=transforms.ToTensor(), # 그냥 받아왔을 때에는 Tensor Value가 아니므로 
                            download=True) 

mnist_test = datasets.MNIST(root='\MNIST_data/',
                           train=False,
                           transform=transforms.ToTensor(),
                           download=True)

Processing...
Done!


In [70]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train, 
                                         batch_size = batch_size, 
                                         shuffle = True, 
                                         drop_last = True)

### Step5: 모델 만들기

In [86]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1), 
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc = nn.Linear(7*7*64, 10, bias=True) # 10: 0~9
        torch.nn.init.xavier_uniform_(self.fc.weight) # 요 Layer만 일단 초기화
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        
        out = out.view(out.size(0), -1) # Batchsize만큼 펼쳐주고 나머지는 알아서 하도록~
        out = self.fc(out)
        
        return out

In [87]:
model = CNN().to(device) # cuda 사용 가능하면 cuda, 안되면 cpu
model

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=3136, out_features=10, bias=True)
)

### Step6: Loss function과 Optimizer 설정하기

In [88]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

### Step7: 학습 및 Loss check

#### CPU 너무 느려서 학습 포기...

In [ ]:
total_batch = len(data_loader)

print('Learning is started. It might take some time.')

for epoch in range(training_epochs):
    avg_cost = 0 # 여기에다가 loss를 하나씩 담을 것 
    
    for X, Y in data_loader:
        # X: image, Y: label
        X = X.to(device) # cuda 연산을 진행하려면 torch.Tensor가 아니라 torch.cuda의 텐서가 되어있어야하기에.
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch 
   
    # 한 Epoch의 학습이 끝나면 Cost 출력
    print('[Epoch: {}] cost = {}'.format(epoch+1, avg_cost))
    
print('Learning Finished!')


Learning is started. It might take some time.
[Epoch: 1] cost = 0.22246138751506805
[Epoch: 2] cost = 0.06280939280986786


### Loss Check

In [ ]:
with torch.no_grad(): # 핛습을 안 할 거니까!
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)  # 불러와서 한 번에 집어 넣으려고 쭉 펼침
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_preddiction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())